### Display points on a base map

In [10]:
import pandas as pd
import folium

# Load the points.csv
# points_df = pd.read_csv('M:\ParFlow\ParFlow_data_extraction_tool\grid_cellscorrect_points.csv')
points_df = pd.read_csv('points_interest.csv')


# Ensure IDs are integers (remove decimals completely)
points_df['cell_id'] = points_df['cell_id'].astype(int) 

# Create a Folium Map centered around the average latitude and longitude
center_lat, center_lon = points_df['lat'].mean(), points_df['lon'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add points as small dots with integer ID labels next to them
for _, row in points_df.iterrows():
    # Add the point as a small dot
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=3,  # Small dot size
        color='blue',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)
    
    # Place the integer ID label next to each point
    folium.Marker(
        location=[row['lat'], row['lon']],
        icon=folium.DivIcon(
            html=f"""<div style="font-size: 10px; color: black; transform: translate(10px, -10px);">{int(row['cell_id'])}</div>"""  # Explicitly ensure ID is displayed as integer
        )
    ).add_to(m)

# Display the map
m


### Extract WTD time series data for the Zwischenscholle Aquifer

In [9]:
import json
import pandas as pd
import numpy as np
import data_extraction_tool
import logging
import os
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to generate the appropriate URL based on type and year
def generate_url(base_url_daily, base_url_monthly, data_type, year):
    start_date = f"{year}0101"
    end_date = f"{year}1231"
    if data_type == "daily":
        return f"{base_url_daily}wtd_DE05_ECMWF-HRES_hindcast_r1i1p2_FZJ-IBG3-ParFlowCLM380_" \
               f"hgfadapter-h00-v02bJurecaGpuProdClimatologyTl_1day_{start_date}-{end_date}.nc"
    elif data_type == "monthly":
        return f"{base_url_monthly}wtd_DE05_ECMWF-HRES_hindcast_r1i1p2_FZJ-IBG3-ParFlowCLM380_" \
               f"hgfadapter-h00-v02bJurecaGpuProdClimatologyTl_1month_{start_date}-{end_date}.nc"
    else:
        raise ValueError("Invalid data type. Choose 'daily' or 'monthly'.")

# Ask user for data type
data_type = input("Enter the type of water table depth data you want to extract (daily/monthly): ").strip().lower()
while data_type not in ["daily", "monthly"]:
    data_type = input("Invalid input. Please enter 'daily' or 'monthly': ").strip().lower()

# Ask user for the year of interest
year = input("Enter the year for data extraction (e.g., 2023): ").strip()
current_year = datetime.now().year
while not year.isdigit() or not (1900 <= int(year) <= current_year):
    year = input(f"Invalid year. Please enter a valid year between 1900 and {current_year}: ").strip()
year = int(year)

# Base URLs
base_url_daily = "https://service.tereno.net/thredds/dodsC/forecastnrw/products/climatology_v2/"
base_url_monthly = "https://service.tereno.net/thredds/dodsC/forecastnrw/products/tmp_isimip_gw/"

# Generate the appropriate URL
url = generate_url(base_url_daily, base_url_monthly, data_type, year)

# Load the points of interest (POIs) data
# points_file_path = r'M:\ParFlow\ParFlow_data_extraction_tool\grid_cells\all_points.csv'
points_file_path = 'points_interest.csv'
if not os.path.exists(points_file_path):
    logging.error(f"Points file not found: {points_file_path}")
    raise FileNotFoundError(f"{points_file_path} not found")

points_data = pd.read_csv(points_file_path)
required_columns = {'cell_id', 'lon', 'lat'}
if not required_columns.issubset(points_data.columns):
    logging.error(f"Missing columns in points.csv. Required: {required_columns}")
    raise ValueError(f"points.csv must contain the columns: {required_columns}")

# Enforce precision of up to 5 decimal places for Latitude and Longitude
points_data['lat'] = points_data['lat'].round(5)
points_data['lon'] = points_data['lon'].round(5)

# Prepare locations within the JSON file
locations = []
for _, row in points_data.iterrows():
    locations.append({
        "locationID": f"Zwischenscholle_Aquifer_Cell_{row['cell_id']}",
        "locationLat": row['lat'],
        "locationLon": row['lon'],
        "simData": url,
        "depth": 0  # Depth is set to 0 for WTD extraction
    })

# Create JSON control file content
json_data = {
    "indicatorFile": "DE-0055_INDICATOR_regridded_rescaled_SoilGrids250-v2017_BGRvector_newAllv.nc",
    "locations": locations
}

'''
# Determine folder name based on year and data type
folder_name = f'wtd_{year}_{data_type}'

# Check if the folder exists, if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
else:
    logging.info(f"Folder already exists: {folder_name}")
'''

# Save JSON control file
json_filename = f'wtd_{year}_{data_type}.json'
with open(json_filename, 'w') as f:
    json.dump(json_data, f, indent=4)
logging.info(f"JSON control file created: {json_filename}")

try:
    # Run data extraction
    data = data_extraction_tool.data_extraction(json_filename, 'var')
    logging.info(f"Data extraction complete. Shape: {data.shape}")
except Exception as e:
    logging.error(f"Data extraction failed: {type(e).__name__} - {e}")
    raise


Enter the type of water table depth data you want to extract (daily/monthly):  monthly
Enter the year for data extraction (e.g., 2023):  2023


2024-11-18 10:48:56,813 - INFO - JSON control file created: wtd_2023_monthly.json


----------------------------------------
JSON record: 0
location ID, lon, lat, depth, sim-data: Zwischenscholle_Aquifer_Cell_1.0, 6.37377, 50.90971, 0, https://service.tereno.net/thredds/dodsC/forecastnrw/products/tmp_isimip_gw/wtd_DE05_ECMWF-HRES_hindcast_r1i1p2_FZJ-IBG3-ParFlowCLM380_hgfadapter-h00-v02bJurecaGpuProdClimatologyTl_1month_20230101-20231231.nc
POI: index x-y and lon-lat on model grid: 641, 1113, 6.3724470138549805, 50.91010665893555
variable name:  wtd
variable long name:  water table depth
variable unit:  m
variable dimension:  3
water table depth is a 2D variable, no depth information needed
timeseries data at location and depth: [3.2298868 3.1102195 3.0522244 2.9262607 2.8419757 2.9062448 3.0388765
 3.1012363 3.1268518 3.1566012 3.0612445 2.7786388]
----------------------------------------
JSON record: 1
location ID, lon, lat, depth, sim-data: Zwischenscholle_Aquifer_Cell_2.0, 6.37582, 50.91095, 0, https://service.tereno.net/thredds/dodsC/forecastnrw/products/tmp_isim

2024-11-18 10:57:14,549 - INFO - Data extraction complete. Shape: (504, 12)


timeseries data at location and depth: [23.896383 23.870535 23.841213 23.80027  23.745848 23.691154 23.654228
 23.635178 23.62404  23.618654 23.613888 23.573708]
----------------------------------------
